<a href="https://colab.research.google.com/github/Matendy12/Project-4/blob/main/Movies_and_Ratings_Deep_Learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'movies-and-ratings-for-recommendation-system:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3747724%2F6486174%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T214144Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3De42a0a0ca0318c3dd704c22d2f6ef7f60b1db72d51d5a17e44111857702a9b842e3f50e664305a06e60e05b027919a1aa5fd9875749204c5f1c595ab1a3f5966a0480145dd3880eb5ee6f6f3c723389106b95d816976137710830655a8a9791d6853ff17fb18513a7169ea4416acbced3d7da38f77bf81165e2c0ce6fed18f1e75af62f5e180997cb1eb94746439ffb007d2535f91b7c62b7ced3fafae73e4f118d802db3997e89cfb79dfb22e3b8825088516eec266dfa521018d1ccd3342a4de58fed99d398537f75feada93d7164ac54405e026c261850fd97eb2e43dfdc8ac0ef9288f729b0b3d615a5e0feed4e3045978a8f2ee8db216426bb5ee11e279'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 866276 bytes downloaded
Downloaded and uncompressed: movies-and-ratings-for-recommendation-system
Data source import complete.


In [ ]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

movies = pd.read_csv("/kaggle/input/movies-and-ratings-for-recommendation-system/movies.csv", index_col="movieId")
ratings = pd.read_csv("/kaggle/input/movies-and-ratings-for-recommendation-system/ratings.csv")

print("Movie Dataset")
display(movies)
print("User Ratings Dataset")
display(ratings)

Movie Dataset


,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


User Ratings Dataset


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [ ]:
# Drop the non-beneficial ID columns, 'timestamp' and 'NAME'.
ratings.drop(['timestamp', 'movieId'], axis=1, inplace=True)
ratings

,userId,rating
0,1,4.0
1,1,4.0
2,1,4.0
3,1,5.0
4,1,5.0
...,...,...
100831,610,4.0
100832,610,5.0
100833,610,5.0
100834,610,5.0


In [ ]:
# Determine the number of unique values in each column.
movies = ratings.dtypes.index.tolist()
ratings[movies].nunique()

userId    610
rating     10
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_type_count = ratings['rating'].value_counts()
app_type_count

rating
4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_count[app_type_count < 600].index)

# Replace in dataframe
for app in application_types_to_replace:
    ratings['rating'] = ratings['rating'].replace(app,"Other")

# Check to make sure binning was successful
ratings['rating'].value_counts()

rating
4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: count, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_count = ratings['rating'].value_counts()
class_count.head()

rating
4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
Name: count, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count = ratings['rating'].value_counts()
class_count_filtered = class_count[class_count > 1]
print(class_count_filtered.head(20))

rating
4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: count, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count < 300].index)

# Replace in dataframe
for cls in classifications_to_replace:
    ratings['rating'] = ratings['rating'].replace(cls,"Other")

# Check to make sure binning was successful
ratings['rating'].value_counts()

rating
4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: count, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_app_df = pd.get_dummies(ratings)
numeric_app_df

,userId,rating
0,1,4.0
1,1,4.0
2,1,4.0
3,1,5.0
4,1,5.0
...,...,...
100831,610,4.0
100832,610,5.0
100833,610,5.0
100834,610,5.0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = numeric_app_df["rating"]
X = numeric_app_df.drop(columns =["rating"],axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
X.head()

,userId
0,1
1,1
2,1
3,1
4,1


In [ ]:
X_train.iloc[0].values

array([232])

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the model - deep neural net
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 =  9
hidden_nodes_layer2 = 18

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 18        
                                                                 
 dense_1 (Dense)             (None, 18)                180       
                                                                 
 dense_2 (Dense)             (None, 1)                 19        
                                                                 
Total params: 217 (868.00 Byte)
Trainable params: 217 (868.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=10)

Epoch 1/10
2364/2364 [==============================] - 5s 2ms/step - loss: -133437.7344 - accuracy: 0.0278
Epoch 2/10
2364/2364 [==============================] - 4s 2ms/step - loss: -297693.2500 - accuracy: 0.0278
Epoch 3/10
2364/2364 [==============================] - 4s 2ms/step - loss: -546807.8750 - accuracy: 0.0278
Epoch 4/10
2364/2364 [==============================] - 5s 2ms/step - loss: -895440.7500 - accuracy: 0.0278
Epoch 5/10
2364/2364 [==============================] - 4s 2ms/step - loss: -1357390.6250 - accuracy: 0.0278
Epoch 6/10
2364/2364 [==============================] - 4s 2ms/step - loss: -1946707.0000 - accuracy: 0.0278
Epoch 7/10
2364/2364 [==============================] - 5s 2ms/step - loss: -2678039.0000 - accuracy: 0.0278
Epoch 8/10
2364/2364 [==============================] - 4s 2ms/step - loss: -3563502.0000 - accuracy: 0.0278
Epoch 9/10
2364/2364 [==============================] - 4s 2ms/step - loss: -4617590.5000 - accuracy: 0.0278
Epoch 10/10
2364/2364 [

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

788/788 - 1s - loss: -6.5458e+06 - accuracy: 0.0280 - 1s/epoch - 1ms/step
Loss: -6545751.5, Accuracy: 0.028005870059132576


In [ ]:
# Export our model to HDF5 file
nn.save('Models/MoviesandRatings.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
